# Simulacija redova i mreža redova
### Mihael Miličević, kolegij Stohastičko modeliranje, ak. god. 2023./2024.

In [1]:
import numpy as np
import math
from distribution_functions import exponential, lognormal
from queues import QueueNetwork

## Kendallova notacija

### $ A/B/s/m $

$ A $ - proces koji generira zahtjeve
$ B $ - proces koji obrađuje zahtjeve
$ s $ - broj poslužitelja
$ m $ - kapacitet spremnika

## $ M/M/\infty $
Sustav bez čekanja i gubitaka.

In [2]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 1

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=np.inf,
    buffer_size=np.inf,
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

ro = consumer_scale / generator_scale

end_time = 100000
queue_network.simulate(end_time)

pi = [np.power(ro, i) * np.exp(-ro) / math.factorial(i) for i in range(6)]
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(6)]

print(f'Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora \n{average_number_of_busy_processors}')
print()

Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.6065306597126334, 0.3032653298563167, 0.07581633246407918, 0.012636055410679864, 0.001579506926334983, 0.0001579506926334983]
Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.6055854545230571, 0.3026170592600327, 0.07708225223472455, 0.012937262947482053, 0.001606610448656382, 0.00015430491923155046]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
0.5
Eksperimentalni rezultat o očekivanom broju zauzetih procesora 
0.5028775429503265


## $ M/M/s/0 $
Sustav s gubitcima

In [3]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 1
s = 3

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=s,
    buffer_size=0
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

pi_0 = [1 / sum([np.power(ro, i) / math.factorial(i) for i in range(s + 1)])]
pi_i = [pi_0[0] * np.power(ro, i) / math.factorial(i) for i in range(1, s + 1)]
pi = pi_0 + pi_i
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(s + 1)]

print(f'Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro * (1 - pi[-1])}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora: \n{average_number_of_busy_processors}')
print()

probability_of_loosing_a_job = queue_network.logger.calculate_probability_of_loosing_a_job(0)
print(f'Teoretski rezultat o vjerojatnosti gubitka posla: \n{pi[-1]}')
print(f'Eksperimentalni rezultat o vjerojatnosti gubitka posla: \n{probability_of_loosing_a_job}')
print()

Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.6075949367088608, 0.3037974683544304, 0.0759493670886076, 0.012658227848101266]
Eksperimentalni rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.6026426884968648, 0.30719905286597526, 0.07730186360950335, 0.012855578959473046]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
0.4936708860759494
Eksperimentalni rezultat o očekivanom broju zauzetih procesora: 
0.5003695169634012

Teoretski rezultat o vjerojatnosti gubitka posla: 
0.012658227848101266
Eksperimentalni rezultat o vjerojatnosti gubitka posla: 
0.012622441666003027


## $ M/M/s/\infty $
Sustav s čekanjem

In [4]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 4
s = 3

# ro < number_of_processors
ro = consumer_scale / generator_scale

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=s,
    buffer_size=np.inf
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

pi_0 = [1 / (sum([np.power(ro, i) / math.factorial(i) for i in range(s)]) + np.power(ro, s) / (math.factorial(s - 1) * (s - ro)))]
pi_i1 = [pi_0[0] * np.power(ro, i) / math.factorial(i) for i in range(1, s)]
pi_i2 = [pi_0[0] * np.power(ro, i) / (math.factorial(s) * np.power(s, i - s)) for i in range(s, 3 * s)]
pi = pi_0 + pi_i1 + pi_i2
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(3 * s)]

print(f'Teoretski rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora: \n{average_number_of_busy_processors}')
print()

probability_of_a_job_waiting = queue_network.logger.calculate_probability_of_a_job_waiting(0, end_time, s)
print(f'Teoretski rezultat o vjerojatnosti čekanja: \n{s / (s - ro) * pi[s]}')
print(f'Eksperimentalni rezultat o vjerojatnosti čekanja: \n{probability_of_a_job_waiting}')
print()

expected_number_of_jobs_in_buffer = queue_network.logger.calculate_expected_number_of_jobs_in_buffer(0, end_time)
print(f'Teoretski rezultat o očekivanom broju poslova u spremniku: \n{ro * s / np.power(s - ro, 2) * pi[s]}')
print(f'Eksperimentalni rezultat o očekivanom broju poslova u spremniku: \n{expected_number_of_jobs_in_buffer}')
print()

expected_waiting_time_in_buffer = queue_network.logger.calculate_expected_waiting_time_in_buffer(0)
print(f'Teoretski rezultat o očekivanom trajanju čekanja u spremniku: \n{s * consumer_scale / np.power(s - ro, 2) * pi[s]}')
print(f'Eksperimentalni rezultat o očekivanom trajanju čekanja u spremniku: \n{expected_waiting_time_in_buffer}')
print()

Teoretski rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.14814814814814814, 0.09876543209876543, 0.06584362139917695, 0.0438957475994513, 0.029263831732967534, 0.01950922115531169]
Eksperimentalni rezultat o stacionarnoj distribuciji  broja poslova u sustavu: 
[0.1112891892128209, 0.22299299817791718, 0.22175187472982943, 0.14693965480307836, 0.0998367901054025, 0.06694759722402509, 0.042575405058393134, 0.027880335422716427, 0.01950880864877568]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
2.0
Eksperimentalni rezultat o očekivanom broju zauzetih procesora: 
1.9983672976571476

Teoretski rezultat o vjerojatnosti čekanja: 
0.4444444444444444
Eksperimentalni rezultat o vjerojatnosti čekanja: 
0.44395685000652485

Teoretski rezultat o očekivanom broju poslova u spremniku: 
0.8888888888888888
Eksperimentalni rezultat o očekivanom broju poslova u spremniku: 
0.8942099638095604
Teoretski rezultat o

## $ M/M/s/m $
Sustav s čekanjem i gubitcima

In [5]:
queue_network = QueueNetwork()

generator_scale = 2
consumer_scale = 4
s = 3
m = 5

ro = consumer_scale / generator_scale

queue = queue_network.create_queue(
    generator_function=exponential(scale=generator_scale),
    consumer_function=exponential(scale=consumer_scale),
    number_of_processors=s,
    buffer_size=m
)

queue_network.set_transition(
    queue=queue,
    transition={
        -1: 1,
        0: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

pi_0 = [1 / (sum([np.power(ro, i) / math.factorial(i) for i in range(s)]) + sum([np.power(ro, i) / (math.factorial(s) * np.power(s, i - s)) for i in range(s, s + m + 1)]))]
pi_i1 = [pi_0[0] * np.power(ro, i) / math.factorial(i) for i in range(1, s)]
pi_i2 = [pi_0[0] * np.power(ro, i) / (math.factorial(s) * np.power(s, i - s)) for i in range(s, s + m + 1)]
pi = pi_0 + pi_i1 + pi_i2
_pi = [queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(s + m + 1)]

print(f'Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{pi}')
print(f'Eksperimentalni rezultat o stacionarnoj distribuciji broja poslova u sustavu: \n{_pi}')
print()

average_number_of_busy_processors = queue_network.logger.calculate_average_number_of_busy_processors(0, end_time)
print(f'Teoretski rezultat o očekivanom broju zauzetih procesora: \n{ro * (1 - pi[-1])}')
print(f'Eksperimentalni rezultat o očekivanom broju zauzetih procesora: \n{average_number_of_busy_processors}')
print()

probability_of_loosing_a_job = queue_network.logger.calculate_probability_of_loosing_a_job(0)
print(f'Teoretski rezultat o vjerojatnosti gubitka posla: \n{pi[-1]}')
print(f'Eksperimentalni rezultat o vjerojatnosti gubitka posla: \n{probability_of_loosing_a_job}')
print()

Teoretski rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.11562252180808881, 0.23124504361617762, 0.23124504361617762, 0.15416336241078507, 0.10277557494052339, 0.06851704996034892, 0.045678033306899284, 0.030452022204599524, 0.02030134813639968]
Eksperimentalni rezultat o stacionarnoj distribuciji broja poslova u sustavu: 
[0.11321756217787897, 0.2286165461379767, 0.23187068236752043, 0.15547419442530327, 0.10351104309546622, 0.06922986003105823, 0.04721938271193136, 0.030025206076801706, 0.020833564390793948]

Teoretski rezultat o očekivanom broju zauzetih procesora: 
1.9593973037272006
Eksperimentalni rezultat o očekivanom broju zauzetih procesora: 
1.9712376630670738

Teoretski rezultat o vjerojatnosti gubitka posla: 
0.02030134813639968
Eksperimentalni rezultat o vjerojatnosti gubitka posla: 
0.021150929842764787


## Mreže redova

### Primjer 1: Proizvodna linija

![](qn1.png)

In [6]:
queue_network = QueueNetwork()

queue1 = queue_network.create_queue(
    generator_function=exponential(scale=3),
    consumer_function=lognormal(mean=1, sigma=1),
    number_of_processors=1,
    buffer_size=5
)

queue2 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=3),
    number_of_processors=1,
    buffer_size=5
)

queue3 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=1),
    number_of_processors=1,
    buffer_size=5
)

queue_network.set_transition(
    queue=queue1,
    transition={
        -1: 0,
        0: 0,
        1: 1,
        2: 0
    }
)

queue_network.set_transition(
    queue=queue2,
    transition={
        -1: 0,
        0: 0,
        1: 0,
        2: 1
    }
)

queue_network.set_transition(
    queue=queue3,
    transition={
        -1: 1,
        0: 0,
        1: 0,
        2: 0
    }
)

end_time = 100000
queue_network.simulate(end_time)

print(f'Stacionarna distribucija za Q1: \n{[queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(7)]}')
print(f'Stacionarna distribucija za Q2: \n{[queue_network.logger.calculate_stationary_distribution(1, end_time, i) for i in range(7)]}')
print(f'Stacionarna distribucija za Q3: \n{[queue_network.logger.calculate_stationary_distribution(2, end_time, i) for i in range(7)]}')
print(f'Stacionarna distribucija za cijelu mrežu: \n{[queue_network.logger.calculate_stationary_distribution_global(end_time, i) for i in range(19)]}')

print(f'Vjerojatnost gubitka posla: \n{queue_network.logger.calculate_probability_of_loosing_a_job_global()}')


Stacionarna distribucija za Q1: 
[0.03948735770805682, 0.05780829253866872, 0.07443593247179442, 0.10282658342303072, 0.14798466621624856, 0.2149467282886909, 0.36246721895953393]
Stacionarna distribucija za Q2: 
[0.381334733685152, 0.21858910398852327, 0.14985815933288898, 0.10674203758318741, 0.07084379355236706, 0.044750932538531026, 0.027795364669844502]
Stacionarna distribucija za Q3: 
[0.792749573277724, 0.16138497922253678, 0.03647280199467507, 0.00761044405789748, 0.0014626219508163056, 0.00022189086750964691, 4.0211557268630714e-05]
Stacionarna distribucija za cijelu mrežu: 
[0.0040774399661311035, 0.013957143314325201, 0.02382377738398646, 0.04088419654216535, 0.0704495253644978, 0.12657015948404157, 0.36701014338769367, 0.16118095837797614, 0.09275636842862597, 0.05285933501639121, 0.02822424191549225, 0.013896068183322145, 0.004174689054158468, 8.020556585785016e-05, 1.2527621358822216e-05, 0.0, 0.0, 0.0, 0.0]
Vjerojatnost gubitka posla: 
0.3764475861031734


## Mreže redova

### Primjer 2: Složeniji primjer

![](qn2.png)

In [7]:
queue_network = QueueNetwork()

queue1 = queue_network.create_queue(
    generator_function=exponential(scale=1),
    consumer_function=exponential(scale=1),
    number_of_processors=3,
    buffer_size=5
)

queue2 = queue_network.create_queue(
    generator_function=exponential(scale=1),
    consumer_function=exponential(scale=1),
    number_of_processors=3,
    buffer_size=5
)

queue3 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=0.5),
    number_of_processors=2,
    buffer_size=3
)

queue4 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=0.5),
    number_of_processors=2,
    buffer_size=3
)

queue5 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=3),
    number_of_processors=5,
    buffer_size=8
)

queue6 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=3),
    number_of_processors=5,
    buffer_size=8
)

queue7 = queue_network.create_queue(
    generator_function=None,
    consumer_function=exponential(scale=0.2),
    number_of_processors=1,
    buffer_size=2
)

queue_network.set_transition(
    queue=queue1,
    transition={
        -1: 0,
        0: 0,
        1: 0,
        2: 0.5,
        3: 0.5,
        4: 0,
        5: 0,
        6: 0
    }
)

queue_network.set_transition(
    queue=queue2,
    transition={
        -1: 0,
        0: 0,
        1: 0,
        2: 0.5,
        3: 0.5,
        4: 0,
        5: 0,
        6: 0
    }
)

queue_network.set_transition(
    queue=queue3,
    transition={
        -1: 0.3,
        0: 0,
        1: 0,
        2: 0,
        3: 0.2,
        4: 0.3,
        5: 0.2,
        6: 0
    }
)

queue_network.set_transition(
    queue=queue4,
    transition={
        -1: 0.3,
        0: 0,
        1: 0,
        2: 0,
        3: 0.2,
        4: 0.2,
        5: 0.3,
        6: 0
    }
)

queue_network.set_transition(
    queue=queue5,
    transition={
        -1: 0,
        0: 0,
        1: 0,
        2: 0.1,
        3: 0.1,
        4: 0,
        5: 0.1,
        6: 0.7
    }
)

queue_network.set_transition(
    queue=queue6,
    transition={
        -1: 0,
        0: 0,
        1: 0,
        2: 0.1,
        3: 0.1,
        4: 0.1,
        5: 0,
        6: 0.7
    }
)

queue_network.set_transition(
    queue=queue7,
    transition={
        -1: 0.4,
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0.2,
        6: 0.4
    }
)

end_time = 100000
queue_network.simulate(end_time)

print(f'Stacionarna distribucija za Q1: \n{[queue_network.logger.calculate_stationary_distribution(0, end_time, i) for i in range(9)]}')
print(f'Stacionarna distribucija za Q2: \n{[queue_network.logger.calculate_stationary_distribution(1, end_time, i) for i in range(9)]}')
print(f'Stacionarna distribucija za Q3: \n{[queue_network.logger.calculate_stationary_distribution(2, end_time, i) for i in range(6)]}')
print(f'Stacionarna distribucija za Q4: \n{[queue_network.logger.calculate_stationary_distribution(3, end_time, i) for i in range(6)]}')
print(f'Stacionarna distribucija za Q5: \n{[queue_network.logger.calculate_stationary_distribution(4, end_time, i) for i in range(14)]}')
print(f'Stacionarna distribucija za Q6: \n{[queue_network.logger.calculate_stationary_distribution(5, end_time, i) for i in range(14)]}')
print(f'Stacionarna distribucija za Q7: \n{[queue_network.logger.calculate_stationary_distribution(6, end_time, i) for i in range(4)]}')
print(f'Stacionarna distribucija za cijelu mrežu: \n{[queue_network.logger.calculate_stationary_distribution_global(end_time, i) for i in range(30)]}')

print(f'Vjerojatnost gubitka posla: \n{queue_network.logger.calculate_probability_of_loosing_a_job_global()}')

Stacionarna distribucija za Q1: 
[0.3634427550415731, 0.3633413370599592, 0.1821643969868358, 0.060419253495080365, 0.020527859974539077, 0.006935422214438916, 0.0021078773218228654, 0.0007935923298548192, 0.0002667971869710834]
Stacionarna distribucija za Q2: 
[0.3629735652228706, 0.36379183946682414, 0.18356667095777485, 0.06073356099670275, 0.020178467069513047, 0.006110773704578181, 0.0018342430922164497, 0.0005980337373460498, 0.00020465741060611038]
Stacionarna distribucija za Q3: 
[0.5333001966895957, 0.32534351356741914, 0.09860449854226414, 0.030493178073016356, 0.00938489644059057, 0.0028710093825354203]
Stacionarna distribucija za Q4: 
[0.3776967384201669, 0.34628323425003377, 0.15740072196171656, 0.07151755103996271, 0.03261599639038669, 0.014482670407535852]
Stacionarna distribucija za Q5: 
[0.07310774557334901, 0.18964351993089817, 0.24537351824812456, 0.21149661782679952, 0.13525766434643513, 0.07007900060500243, 0.03649224441877264, 0.01917927533977697, 0.00960803738157